# Component Graphs

EvalML component graphs represent and describe the flow of data in a collection of related components. A component graph is comprised of nodes representing components, and edges between pairs of nodes representing where the input and output of one component should go. It is the backbone of the features offered by the EvalML pipeline [user_guide/pipelines.ipynb], but is also a powerful data structure on its own. EvalML currently supports component graphs as linear and directed acyclic graphs (DAG).

## Initalizing a component graph

Component graphs can be initalized by specifying the dictionary of components and edges that describe the graph. In this dictionary, each key is the name that should be used to reference a component by, and each corresponding value is a list where the first element is the component or component name, and the remaining elements are the input edges that should be connected to that component.


For example, in the code example below, we have a simple component graph made up of two components: an Imputer and a Random Forest Classifer. The names used to reference these two components are given by the keys, "My Imputer" and "RF Classifier". Each value in the dictionary is a list where the first element is the component corresponding to the key, and the remaining elements are the inputs. For example, "My Imputer" represents an Imputer component which is fed "X" and "y", the original features and target. Each component can have a number of feature inputs (identified as "X" or ending in ".x") but can only have one target input (identified as "y" or ending in ".y"). All input edges must be explicitly defined.

In [1]:
component_dict = {
    'My Imputer': ['Imputer', 'X', 'y'],
    'RF Classifier': ['Random Forest Classifier', 'My Imputer.x', 'y']
}
ComponentGraph(component_)

A key note is that it must end in one component

In [2]:
component_dict = {
    'My Imputer': ['Imputer', 'X', 'y'],
    'RF Classifier': ['Random Forest Classifier', 'My Imputer.x', 'y'],
    'EN Classifier': ['Elastic Net Classifier', 'My Imputer.x', 'y']
    
}

SyntaxError: invalid syntax (<ipython-input-2-76a5e5c21256>, line 4)

## Get component

We can get a component by its name using the convenient `get_component` method

## Component graph methods

Similar to the pipeline structure, we can call `fit`, `transform` and `predict`. We can also call `fit_features` which will fit all but the final component. This is useful in cases where you want to understand what transformed features are being fed into the last component, which is usually an estimator or model of interest.


required to have one final component.

get_component

similar to pipelines can fit/predict/etc

.graph

(`c0`, `c1`), 
component dictionaries: used to initalize the graph.